# Data Read and Formatting

Notebook para la lectura de los datos y para su modificación de formato en archivos csv, con tablas individuales.

## Path Configuration

In [38]:
import os
from pathlib import Path
print("Directorio actual:", os.getcwd())

current_dir = Path.cwd()
project_root = current_dir
while not (project_root / '.git').exists() and project_root != project_root.parent:
    project_root = project_root.parent
    os.chdir("..")

print(f"Raíz del proyecto: {project_root}")
print("CWD cambiado a raíz del proyecto")

Directorio actual: c:\Users\santi\OneDrive\Documents\GitHub\PROYECTO_MA2003B
Raíz del proyecto: c:\Users\santi\OneDrive\Documents\GitHub\PROYECTO_MA2003B
CWD cambiado a raíz del proyecto


In [39]:
import pandas as pd

path_2024 = r"data/raw/DATOS HISTÓRICOS 2024_TODAS ESTACIONES.xlsx"
excel_file_2024 = pd.ExcelFile(path_2024)
sheet_names_2024 = excel_file_2024.sheet_names
df_2024 = pd.read_excel(path_2024,
                   sheet_name = sheet_names_2024)

In [40]:
df_2024 # Mostrar las primeras filas del DataFrame para verificar la carga correcta

{'SE':             Fecha y hora  CO (ppm)  NO (ppb)  NO2 (ppb)  NOX (ppb)  O3 (ppb)  \
 0    2024-01-01 00:00:00      2.08      10.5       23.3       33.8      13.0   
 1    2024-01-01 01:00:00      2.07      10.5       23.4       33.9      13.0   
 2    2024-01-01 02:00:00      2.36      16.9       30.0       46.9       7.0   
 3    2024-01-01 03:00:00      2.11      13.6       26.6       40.1       7.0   
 4    2024-01-01 04:00:00      1.94       8.5       20.6       29.2      13.0   
 ...                  ...       ...       ...        ...        ...       ...   
 8779 2024-12-31 19:00:00      0.20       2.8        7.6       10.4      55.0   
 8780 2024-12-31 20:00:00      0.20       2.6        6.9        9.4      62.0   
 8781 2024-12-31 21:00:00      0.21       2.6       12.9       15.5      56.0   
 8782 2024-12-31 22:00:00      0.20       2.7       16.0       18.6      47.0   
 8783 2024-12-31 23:00:00      0.19       2.6       10.5       13.0      44.0   
 
       PM10 (ug/m3) 

In [41]:
import pandas as pd

hojas = df_2024

mapeo_nombres = {
    "SE": "sureste",
    "CE": "centro",
    "NO": "noroeste",
    "NE": "noreste",
    "NTE": "norte",
    "SO": "suroeste",
    "SUR": "sur",
    "NE2": "noreste 2",
    "NO2": "noroeste 2",
    "SE2": "sureste 2",
    "NTE2": "norte 2",
    "SO2": "suroeste 2", 
    "SE3": "sureste 3",
    "NE3": "noreste 3",
    "NO3": "noroeste 3"
}

# Lista para guardar los DataFrames concatenados
dfs = []

for nombre_hoja, df in hojas.items():
    # Poner todos los nombres de columnas en minúsculas
    df.columns = df.columns.str.lower()
    
    # Cambiar "date" a "date_index" si existe
    if "fecha y hora" in df.columns:
        df = df.rename(columns={"fecha y hora": "date"})
    
    # Usar el mapeo para obtener el nuevo nombre, si no existe, usa el original
    nombre_estacion = mapeo_nombres.get(nombre_hoja, nombre_hoja)
    # Agregar la columna 'estacion'
    df["estacion"] = nombre_estacion
    
    # Limpiar caracteres no imprimibles de strings
    df = df.applymap(
        lambda x: "".join(ch for ch in str(x) if ch.isprintable()).strip() if isinstance(x, str) else x
    )
    
    # Excluir "CATÁLOGO" de la unión
    if nombre_hoja != "CATÁLOGO":
        dfs.append(df)

# Concatenar todas las hojas excepto "CATÁLOGO"
df_final = pd.concat(dfs, ignore_index=True)

# Guardar en un nuevo archivo con una sola hoja (usando xlsxwriter)
nuevo_archivo = "DATOS_HISTORICOS_2024_CONCATENADO.csv"
df_final.to_csv(nuevo_archivo, index=False)

print("Archivo guardado como:", nuevo_archivo)

C:\Users\santi\AppData\Local\Temp\ipykernel_29724\98329173.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(
C:\Users\santi\AppData\Local\Temp\ipykernel_29724\98329173.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(
C:\Users\santi\AppData\Local\Temp\ipykernel_29724\98329173.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(
C:\Users\santi\AppData\Local\Temp\ipykernel_29724\98329173.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(
C:\Users\santi\AppData\Local\Temp\ipykernel_29724\98329173.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(
C:\Users\santi\AppData\Local\Temp\ipykernel_29724\98329173.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.ap

Archivo guardado como: DATOS_HISTORICOS_2024_CONCATENADO.csv
